In [1]:
# Importando as bibliotecas 
import numpy as np
import pandas as pd
import twitter
import json
import nltk
from sklearn.model_selection import train_test_split

%matplotlib inline
 

In [2]:
df_twitter = pd.read_csv("BaseTwitter -ORIGINAL.csv",sep=",",encoding="ISO-8859-1")

In [3]:
base_treinamento, base_teste=train_test_split(df_twitter)  #75% para treino e 25% para teste

In [4]:
# Transforma base em pandas
Analise_Treinamento=pd.DataFrame(base_treinamento)

#Dá nome as colunas
Analise_Treinamento.columns=['Frase','Sentimento']

# Mostra tamanho da base de treinamento
print('Tamanho da base de Treinamento {}'.format(Analise_Treinamento.shape[0]))

# Mostra quantidade de cada tipo de sentimento
Analise_Treinamento.Sentimento.value_counts()

Tamanho da base de Treinamento 2998


alegria     1583
tristeza    1415
Name: Sentimento, dtype: int64

In [5]:
# Percentual de cada tipo de sentimento
print((Analise_Treinamento.Sentimento.value_counts()/Analise_Treinamento.shape[0])*100)

alegria     52.801868
tristeza    47.198132
Name: Sentimento, dtype: float64


In [6]:
# Criar base teste
# Transforma base em pandas
Analise_Teste=pd.DataFrame(base_teste)

#Dá nome as colunas
Analise_Teste.columns=['Frase','Sentimento']

# Mostra tamanho da base de treinamento
print('Tamanho da base de Teste {}'.format(Analise_Teste.shape[0]))

# Mostra quantidade de cada tipo de sentimento
Analise_Teste.Sentimento.value_counts()

Tamanho da base de Teste 1000


alegria     503
tristeza    497
Name: Sentimento, dtype: int64

In [7]:
#nltk.download('stopwords')
lista_Stop=nltk.corpus.stopwords.words('portuguese')
np.transpose(lista_Stop)

array(['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para',
       'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as',
       'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou',
       'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo',
       'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo',
       'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa',
       'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos',
       'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas',
       'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus',
       'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos',
       'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele',
       'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está',
       'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram',
       'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos

In [8]:
# Função que retira StopWords
def RemoveStopWords(texto):
    frases=[]
    for (palavras, sentimento) in texto:
        #Criamos uma list compreheension para extrair apenas as palavras que não estão na list_stop
        semStop= [ p for p in palavras.split() if p not in lista_Stop]
        #Inserindo as frases com os Labels (sentimento) ja tatadas pela Lista_Stop
        frases.append((semStop,sentimento))
    return frases

In [9]:
# Steemer= técnica de remover sufixos e prefixos de uma palavra,Por exemplo, o stem da palavra cooking é cook. Um bom algoritmo sabe que “ing” é um sufixo e pode ser removido.
#nltk.download('rslp')
def aplica_Stemmer(texto):
    stemmer=nltk.stem.RSLPStemmer()
    #Escolhido o RSLPS pois é especifico da língua portuguesas
    frases_sem_Stemming=[]
    for (palavras, sentimento) in texto:
        com_Stemming=[str(stemmer.stem(p)) for p in palavras.split() if p not in lista_Stop]
        frases_sem_Stemming.append((com_Stemming,sentimento))
    return frases_sem_Stemming

In [10]:
#Transforma base de treinamento em um array
base_treinamento=np.array(base_treinamento)

In [11]:
# Aplica o Stemer na Frase
# Transformar em vetor está dando problema
frases_com_Stem_treinamento=aplica_Stemmer(base_treinamento)

In [12]:
#Transforma base de teste em um array
base_teste=np.array(base_teste)

In [13]:
# Aplica Stem na base de teste
frases_com_Stem_teste=aplica_Stemmer(base_teste)

In [14]:
# Cria função para retornar as palavras da frase, sem a classificação(sentimento)
def busca_Palavras(frases):
    todas_Palavras=[]
    for (palavras,sentimento) in frases:
        todas_Palavras.extend(palavras)
    return todas_Palavras

In [15]:
# Aplica função acima em cada base
palavras_treinamento=busca_Palavras(frases_com_Stem_treinamento)
palavras_teste=busca_Palavras(frases_com_Stem_teste)

In [16]:
# Mostra quantidade de palavras na base
print ("Quantidade de palavras Treinamento {}".format(pd.DataFrame(palavras_treinamento).count()))
print ("Quantidade de palavras Teste {}".format(pd.DataFrame(palavras_teste).count()))

Quantidade de palavras Treinamento 0    37342
dtype: int64
Quantidade de palavras Teste 0    12622
dtype: int64


In [17]:
# Função para verificar a quantidade de vezes que a palavras é mencionada
def busca_frequencia(palavras):
    palavras=nltk.FreqDist(palavras)
    return palavras

In [18]:
# Chama função acima
frequencia_treinamento=busca_frequencia(palavras_treinamento)

In [19]:
# A função most_common do NLTK possibilita visualizar quais as palavras que ocorrem com maior frequência em nosso texto.
frequencia_treinamento.most_common(20)

[('iphon', 2936),
 ('compr', 568),
 ('quebr', 484),
 ('troq', 434),
 ('ganh', 434),
 ('estrag', 389),
 ('nã£', 385),
 ('molh', 363),
 ('pra', 340),
 ('q', 331),
 ('celul', 308),
 ('consert', 305),
 ('meu', 296),
 ('caiu', 290),
 ('ã©', 258),
 ('iphone,', 246),
 ('eu', 237),
 (',', 183),
 ('arrum', 182),
 ('to', 182)]

In [20]:
# Executa função parecida com a mostcommon
frequencia_teste=busca_frequencia(palavras_teste)

In [21]:
# Função para retornar somente as palavras unicas
def busca_palavras_unicas(frequencia):
    freq=frequencia.keys()
    return freq

# Chama função acima
palavras_unicas_treinamento=busca_palavras_unicas(frequencia_treinamento)
palavras_unicas_teste=busca_palavras_unicas(frequencia_teste)

In [22]:
# Cria função para identificar quais as palavras únicas estão no docuemnto passado para a função
def extrator_palavras(documento):
        #Utilizado set() para associar a variável doc.com o parametro que esta chegando
        doc=set(documento)
        caracteristicas={}
        for palavras in palavras_unicas_treinamento:
            caracteristicas['%s' % palavras]=(palavras in doc)
        return caracteristicas

In [23]:
# devido a necessidade de aplciação da função Extrato_Palavras sobre as bases de Treinamento e Teste, como a variável palavras_unicas_teste exige aplicação isolada, precisamos criar uma função apartada somente
# para a base de teste
def extrator_palavras_teste(documento):
    doc=set(documento)
    caracteristicas={}
    for palavras in palavras_unicas_teste:
        caracteristicas['%s' % palavras]=(palavras in doc)
    return caracteristicas

In [24]:
base_completa_treinamento=nltk.classify.apply_features(extrator_palavras, frases_com_Stem_treinamento)
base_completa_teste=nltk.classify.apply_features(extrator_palavras_teste,frases_com_Stem_teste)

In [25]:
# O algoritmo NaiveBayes monta a tabela de probabilidade
classificador=nltk.NaiveBayesClassifier.train(base_completa_treinamento)
print(classificador.labels())

['alegria', 'tristeza']


In [26]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                       @ = True           triste : alegri =     64.5 : 1.0
                    caiu = True           triste : alegri =     44.8 : 1.0
                  estrag = True           triste : alegri =     41.6 : 1.0
                 consert = True           alegri : triste =     22.3 : 1.0
                       . = True           triste : alegri =     19.9 : 1.0
                       ' = True           triste : alegri =     19.8 : 1.0
                     cha = True           triste : alegri =     19.8 : 1.0
                  derrub = True           triste : alegri =     17.7 : 1.0
                     ess = True           triste : alegri =     15.4 : 1.0
                    troq = True           alegri : triste =     13.1 : 1.0
None


In [27]:
# Verifica a acuracia do modelo com a base de treinamento
print(nltk.classify.accuracy(classificador,base_completa_treinamento))

0.9829886591060707


In [28]:
# Agrega linhas com erros de classificação
erros=[]
for (frase,classe) in base_completa_treinamento:
    #print(frase)
    #print(classe)
    resultado=classificador.classify(frase)
    if resultado != classe:
        erros.append((classe,resultado,frase))

In [29]:
# Cria matriz de confusão para de treinamento
from nltk.metrics import ConfusionMatrix
esperado=[]
previsto=[]
for teste in base_completa_treinamento:
    resultado=classificador.classify(teste[0])
    previsto.append(resultado)
    esperado.append(teste[1])

matriz=ConfusionMatrix(esperado,previsto)
print (matriz)



         |         t |
         |    a    r |
         |    l    i |
         |    e    s |
         |    g    t |
         |    r    e |
         |    i    z |
         |    a    a |
---------+-----------+
 alegria |<1571>  12 |
tristeza |   39<1376>|
---------+-----------+
(row = reference; col = test)



In [30]:
# Cria matriz de confusão para de teste
from nltk.metrics import ConfusionMatrix
esperado=[]
previsto=[]
for teste in base_completa_teste:
    resultado=classificador.classify(teste[0])
    previsto.append(resultado)
    esperado.append(teste[1])
matriz=ConfusionMatrix(esperado,previsto)

print (matriz)



         |       t |
         |   a   r |
         |   l   i |
         |   e   s |
         |   g   t |
         |   r   e |
         |   i   z |
         |   a   a |
---------+---------+
 alegria |<487> 16 |
tristeza |  21<476>|
---------+---------+
(row = reference; col = test)



In [32]:
teste="Comprei um celular novo!"
testeStemming=[]
stemmer=nltk.stem.RSLPStemmer()
for (palavras_treinamento) in teste.split():
    comStem=[p for p in palavras_treinamento.split()]
    testeStemming.append(str(stemmer.stem(comStem[0])))

novo=extrator_palavras(testeStemming)
#print(classificador.classify(novo))
distribuicao = classificador.prob_classify(novo)
resultado_final=classificador.classify(novo)
for classe in distribuicao.samples():
    print('%s: %f' % (classe, distribuicao.prob(classe)))
    
print("Resultado final: "+ resultado_final)
    

alegria: 0.722784
tristeza: 0.277216
Resultado final: alegria
